In [1]:
import os
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)
import geopandas as gpd
import numpy as np
import math

In [2]:
# data gathered on 19th of Feb, 2021
INFO_PATH = "../data/country_info.csv"  # https://www.kaggle.com/koryto/countryinfo?select=covid19countryinfo.csv
VACCINATION_PATH = "../data/country_vaccinations.csv"  # https://www.kaggle.com/gpreda/covid-world-vaccination-progress
SHAPEFILE_PATH = "../shapefiles/ne_10m_admin_0_countries_lakes/ne_10m_admin_0_countries_lakes.shp"  # https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries_lakes.zip

In [3]:
geo_df = gpd.read_file(SHAPEFILE_PATH)[['ADM0_A3', 'geometry']].to_crs('+proj=robin')
info_df = pd.read_csv(INFO_PATH, usecols=["country", "alpha3code", "pop"])
info_df = info_df[(~info_df['alpha3code'].isna()) & (~info_df['pop'].isna())]
info_df['pop'] = info_df['pop'].str.replace(",", "").astype(int)
vac_df = pd.read_csv(VACCINATION_PATH, usecols=["country", "iso_code", "date", "total_vaccinations","daily_vaccinations"], parse_dates=["date"])
vac_df = vac_df[~vac_df['iso_code'].isna()]

In [4]:
vac_df = vac_df[vac_df['iso_code'].isin(info_df['alpha3code'].tolist())]
vac_df.tail()

country iso_code       date  total_vaccinations  \
3325  United States      USA 2021-02-13          50641884.0   
3326  United States      USA 2021-02-14          52884356.0   
3327  United States      USA 2021-02-15                 NaN   
3328  United States      USA 2021-02-16          55220364.0   
3329  United States      USA 2021-02-17          56281827.0   

      daily_vaccinations  
3325           1657703.0  
3326           1667631.0  
3327           1662106.0  
3328           1716311.0  
3329           1644551.0

In [5]:
all_countries = vac_df[vac_df['daily_vaccinations'].isna() == False].country.unique().tolist()
good_confidence_countries = []
countries_daily_fforward = {}
countries_first_update = {}
countries_latest_update = {}

for c in all_countries:
    first_non_na_i = vac_df[(vac_df['daily_vaccinations'].isna() == False) & (vac_df['country'] == c)].head(1).index.item()
    this_c_df = vac_df[(vac_df.index >= first_non_na_i) & (vac_df.country == c)]
    nans_after_first_nan = this_c_df.daily_vaccinations.isna().sum()
    days_with_info = this_c_df.daily_vaccinations.notna().sum()
    first_non_nan_update = this_c_df[this_c_df.daily_vaccinations.notna()]['date'].head(1).item()
    latest_non_nan_update = this_c_df[this_c_df.daily_vaccinations.notna()]['date'].tail(1).item()
    lastest_date = this_c_df['date'].tail(1).item()
    if days_with_info < 15:
        continue
    else:
        good_confidence_countries.append(c)
        print(f"{c}: nans after first non nan: {nans_after_first_nan}, number of days with info: {days_with_info}, first and last update: {first_non_nan_update} & {latest_non_nan_update}")
        all_time_mean = this_c_df[this_c_df.daily_vaccinations.notna()]['daily_vaccinations'].mean()
        recent_mean = this_c_df[this_c_df.daily_vaccinations.notna()]['daily_vaccinations'].tail(5).mean()
        selected_forward_daily_vac = math.floor(0.15 * all_time_mean + recent_mean * 0.85)
        countries_daily_fforward[c] = selected_forward_daily_vac
        countries_first_update[c] = first_non_nan_update
        countries_latest_update[c] = latest_non_nan_update
        
vac_df = vac_df[vac_df['country'].isin(good_confidence_countries)]

Albania: nans after first non nan: 6, number of days with info: 32, first and last update: 2021-01-11 00:00:00 & 2021-02-17 00:00:00
Andorra: nans after first non nan: 0, number of days with info: 18, first and last update: 2021-01-26 00:00:00 & 2021-02-12 00:00:00
Argentina: nans after first non nan: 0, number of days with info: 50, first and last update: 2020-12-30 00:00:00 & 2021-02-17 00:00:00
Austria: nans after first non nan: 0, number of days with info: 35, first and last update: 2021-01-14 00:00:00 & 2021-02-17 00:00:00
Azerbaijan: nans after first non nan: 0, number of days with info: 20, first and last update: 2021-01-18 00:00:00 & 2021-02-06 00:00:00
Bahrain: nans after first non nan: 0, number of days with info: 56, first and last update: 2020-12-24 00:00:00 & 2021-02-17 00:00:00
Bangladesh: nans after first non nan: 0, number of days with info: 22, first and last update: 2021-01-27 00:00:00 & 2021-02-17 00:00:00
Belgium: nans after first non nan: 0, number of days with inf

In [6]:
vac_df[vac_df['date'] == vac_df[vac_df.daily_vaccinations.notna()]['date'].min()]

country iso_code       date  total_vaccinations  \
3205  United Kingdom      GBR 2020-12-14                 NaN   

      daily_vaccinations  
3205             83940.0

In [7]:
def create_country_date_df(countries, start, end):
    all_dfs = []
    for country in countries:
        df = pd.DataFrame({"date": pd.date_range(start, end), "country": country})
        all_dfs.append(df)
    concat_df = pd.concat(all_dfs, ignore_index=True)
    return concat_df

In [8]:
clean_df =  create_country_date_df(good_confidence_countries, '2020-12-14', '2022-12-31')

In [9]:
clean_df.loc[745:,:].head()

date  country
745 2022-12-29  Albania
746 2022-12-30  Albania
747 2022-12-31  Albania
748 2020-12-14  Andorra
749 2020-12-15  Andorra

In [10]:
clean_df = pd.merge(left=clean_df, right=vac_df, how='left', on=['date','country'])

In [11]:
clean_df["iso_code"] = clean_df.groupby("country").transform(lambda x: x.fillna(method="bfill"))["iso_code"]
clean_df["iso_code"] = clean_df.groupby("country").transform(lambda x: x.fillna(method="ffill"))["iso_code"]
clean_df.loc[745:,:].head()

date  country iso_code  total_vaccinations  daily_vaccinations
745 2022-12-29  Albania      ALB                 NaN                 NaN
746 2022-12-30  Albania      ALB                 NaN                 NaN
747 2022-12-31  Albania      ALB                 NaN                 NaN
748 2020-12-14  Andorra      AND                 NaN                 NaN
749 2020-12-15  Andorra      AND                 NaN                 NaN

In [12]:
clean_df = clean_df.merge(info_df[["alpha3code", "pop"]], how="left", left_on="iso_code", right_on="alpha3code").drop(['alpha3code'], axis=1).rename({"pop":"needed_vaccines"}, axis=1)
clean_df['needed_vaccines'] = clean_df['needed_vaccines'].astype(np.int64) * 2 # converting from numpy.int32 to numpy.int64 to prevent overflow

In [13]:
def correct_daily_vac_col(x):
    dt = x['date']
    country = x['country']
    before_first_info = dt < countries_first_update[country]
    after_last_info = dt > countries_latest_update[country]
    if before_first_info:
        return 0
    elif after_last_info:
        return countries_daily_fforward[country]
    elif pd.isnull(x['daily_vaccinations']):
        return x['daily_vaccinations']
    else:
        return math.floor(x['daily_vaccinations'])
clean_df['daily_vaccinations'] = clean_df.apply(lambda x: correct_daily_vac_col(x), axis=1)
clean_df['daily_vaccinations'] = clean_df['daily_vaccinations'].fillna(method="ffill")

In [14]:
clean_df[(clean_df['country'] == "Albania") & (clean_df['date'] > '2021-01-09') & (clean_df['date'] < '2021-02-20')]

date  country iso_code  total_vaccinations  daily_vaccinations  \
27 2021-01-10  Albania      ALB                 0.0                 0.0   
28 2021-01-11  Albania      ALB                 NaN                64.0   
29 2021-01-12  Albania      ALB               128.0                64.0   
30 2021-01-13  Albania      ALB               188.0                63.0   
31 2021-01-14  Albania      ALB               266.0                66.0   
32 2021-01-15  Albania      ALB               308.0                62.0   
33 2021-01-16  Albania      ALB               369.0                62.0   
34 2021-01-17  Albania      ALB               405.0                58.0   
35 2021-01-18  Albania      ALB               447.0                55.0   
36 2021-01-19  Albania      ALB               483.0                51.0   
37 2021-01-20  Albania      ALB               519.0                47.0   
38 2021-01-21  Albania      ALB               549.0                40.0   
39 2021-01-22  Albania      ALB                 NaN                34.0   
40 2021-01-23  Albania      ALB                 NaN                26.0   
41 2021-01-24  Albania      ALB                 NaN                21.0   
42 2021-01-25  Albania      ALB                 NaN                15.0   
43 2021-01-26  Albania      ALB                 NaN                 9.0   
44 2021-01-27  Albania      ALB                 NaN                 4.0   
45 2021-01-28  Albania      ALB                 NaN                 4.0   
46 2021-01-29  Albania      ALB                 NaN                 4.0   
47 2021-01-30  Albania      ALB                 NaN                 4.0   
48 2021-01-31  Albania      ALB                 NaN                 4.0   
49 2021-02-01  Albania      ALB                 NaN                 4.0   
50 2021-02-02  Albania      ALB               550.0                 4.0   
51 2021-02-03  Albania      ALB                 NaN                12.0   
52 2021-02-04  Albania      ALB                 NaN                24.0   
53 2021-02-05  Albania      ALB                 NaN                35.0   
54 2021-02-06  Albania      ALB                 NaN                47.0   
55 2021-02-07  Albania      ALB                 NaN                59.0   
56 2021-02-08  Albania      ALB                 NaN                71.0   
57 2021-02-09  Albania      ALB              1127.0                82.0   
58 2021-02-10  Albania      ALB                 NaN                81.0   
59 2021-02-11  Albania      ALB                 NaN                79.0   
60 2021-02-12  Albania      ALB                 NaN                78.0   
61 2021-02-13  Albania      ALB                 NaN                76.0   
62 2021-02-14  Albania      ALB                 NaN                75.0   
63 2021-02-15  Albania      ALB                 NaN                73.0   
64 2021-02-16  Albania      ALB                 NaN                72.0   
65 2021-02-17  Albania      ALB              1701.0                72.0   
66 2021-02-18  Albania      ALB                 NaN                70.0   
67 2021-02-19  Albania      ALB                 NaN                70.0   

    needed_vaccines  
27          5755594  
28          5755594  
29          5755594  
30          5755594  
31          5755594  
32          5755594  
33          5755594  
34          5755594  
35          5755594  
36          5755594  
37          5755594  
38          5755594  
39          5755594  
40          5755594  
41          5755594  
42          5755594  
43          5755594  
44          5755594  
45          5755594  
46          5755594  
47          5755594  
48          5755594  
49          5755594  
50          5755594  
51          5755594  
52          5755594  
53          5755594  
54          5755594  
55          5755594  
56          5755594  
57          5755594  
58          5755594  
59          5755594  
60          5755594  
61          5755594  
62          5755594  
63          5755594  
64          5755594  


In [15]:
clean_df["total_vaccinations"] = clean_df.groupby(['country'])['daily_vaccinations'].cumsum(axis=0)

In [16]:
clean_df["immunization_coverage"] = np.where((clean_df.total_vaccinations / clean_df.needed_vaccines) < 1, (clean_df.total_vaccinations / clean_df.needed_vaccines), 1)
clean_df['immunization_coverage'] = clean_df['immunization_coverage'] * 100

In [17]:
geo_df.head()

ADM0_A3                                           geometry
0     IDN  MULTIPOLYGON (((11108970.260 445285.130, 11108...
1     MYS  MULTIPOLYGON (((11108970.260 445285.130, 11108...
2     CHL  MULTIPOLYGON (((-6477059.899 -1872360.838, -64...
3     BOL  POLYGON ((-6477059.899 -1872360.838, -6477072....
4     PER  MULTIPOLYGON (((-6477059.899 -1872360.838, -64...

In [18]:
merged = pd.merge(geo_df, clean_df, how='left', left_on='ADM0_A3', right_on='iso_code').drop(["iso_code"], axis=1)

In [20]:
title = "      COVID-19 World Immunization Progress Until 2023"
description = "If vaccination across various countries remain in the same pace(as on February 19th, 2021), thats how the race goes.\n"\
"                   Data: Our World in Data - ourworldindata.org • Author: Gabriel Tardochi Salles".strip()
# set a variable that will call whatever column we want to visualise on the map
variable = 'immunization_coverage'
# set the range for the choropleth
vmin, vmax = 0, 100

completely_immunized = []
ranking = ''
# creating the img files
for date in clean_df.date.unique():
    this_plot = merged[(merged['date'] == date) | (merged['date'].isna())]
    date_time = pd.to_datetime(str(date)) 
    date_string_to_plot = date_time.strftime("%b, %Y")
    date_string_to_save = date_time.strftime("%Y%m%d")
    month_year_only = date_time.strftime("%b/%Y")
    saving_path = f"../out/images/img_{date_string_to_save}.jpg"
    
    # checking if someone got 100%
    now_completed_countries = this_plot[(this_plot['immunization_coverage'] == 100) & ~(this_plot['country'].isin(completely_immunized))]['country'].tolist()
    for country in now_completed_countries:
        completely_immunized.append(country)
        podium_len = len(completely_immunized)
        suf = lambda n: "%d%s"%(n,{1:"st",2:"nd",3:"rd"}.get(n if n<20 else n%10,"th"))
        ranking += f"{suf(podium_len)} - {country} ({month_year_only})\n"
    if date_string_to_save == "20221231":
        ranking += "                . . ."
#     if not (date_string_to_save.endswith("05") or date_string_to_save.endswith("15") or date_string_to_save.endswith("25")) or date_string_to_save == "20221231":
#         continue
    # create figure and axes for Matplotlib
    if not date_string_to_save == "20221231":
        continue
    fig, ax = plt.subplots(1, figsize=(14, 7))
    this_plot.dropna().plot(column=variable, cmap='Purples', linewidth=0.8, ax=ax, edgecolor='.9', vmin=vmin, vmax=vmax, legend=False, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    # remove the axis
    ax.axis('off')
    # add a title
    ax.set_title(title, fontdict={'fontsize': '24', 'fontweight' : '2'}, color='#0A0122')
    # create an annotation for the data source
    ax.annotate(description, xy=(0.1, .065), xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
    # Create colorbar as a legend
    sm = plt.cm.ScalarMappable(cmap='Purples', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    # empty array for the data range
    sm._A = []
    # add the colorbar to the figure
    cbar = fig.colorbar(sm, label="% of the Population Fully Immunized", pad=0, shrink=0.7)
    plt.subplots_adjust(left=-2, right=2, bottom=-1, top=2)
    # position the annotation to the bottom left
    ax.annotate(date_string_to_plot,
            xy=(0.14, .265), xycoords='figure fraction',
            horizontalalignment='center', verticalalignment='top',
            color='#3C4052', fontsize=20)

    ax.annotate(ranking,
            xy=(0.032, .57), xycoords='figure fraction', color='#717171',
            horizontalalignment='left', verticalalignment='top',
            fontsize=11)
    this_plot[this_plot.isna().any(axis=1)].plot(ax=ax, color='#fafafa', hatch='///')
    fig.tight_layout(pad=0.05)
    plt.savefig(saving_path, dpi=300)
    if date_string_to_save == "20221231":
        for i in range(10):
            plt.savefig(f"../out/images/img_{date_string_to_save + str(i)}.jpg", dpi=300)
    plt.close('all')